In [1]:
import binascii, urllib, socket, random, struct
from bcode import bdecode
from urlparse import urlparse, urlunsplit

def scrape(tracker, hashes):
    """
    Returns the list of seeds, peers and downloads a torrent info_hash has, according to the specified tracker
    Args:
        tracker (str): The announce url for a tracker, usually taken directly from the torrent metadata
        hashes (list): A list of torrent info_hash's to query the tracker for
    Returns:
        A dict of dicts. The key is the torrent info_hash's from the 'hashes' parameter,
        and the value is a dict containing "seeds", "peers" and "complete".
        Eg:
        {
            "2d88e693eda7edf3c1fd0c48e8b99b8fd5a820b2" : { "seeds" : "34", "peers" : "189", "complete" : "10" },
            "8929b29b83736ae650ee8152789559355275bd5c" : { "seeds" : "12", "peers" : "0", "complete" : "290" }
        }
    """
    tracker = tracker.lower()
    parsed = urlparse(tracker)	
    if parsed.scheme == "udp":
        return scrape_udp(parsed, hashes)

    if parsed.scheme in ["http", "https"]:
        if "announce" not in tracker:
            raise RuntimeError("%s doesnt support scrape" % tracker)
        parsed = urlparse(tracker.replace("announce", "scrape"))		 
        return scrape_http(parsed, hashes)

    raise RuntimeError("Unknown tracker scheme: %s" % parsed.scheme)	

def scrape_udp(parsed_tracker, hashes):
    print "Scraping UDP: %s for %s hashes" % (parsed_tracker.geturl(), len(hashes))
    if len(hashes) > 74:
        raise RuntimeError("Only 74 hashes can be scraped on a UDP tracker due to UDP limitations")
    transaction_id = "\x00\x00\x04\x12\x27\x10\x19\x70";
    connection_id = "\x00\x00\x04\x17\x27\x10\x19\x80";
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.settimeout(8)
    conn = (socket.gethostbyname(parsed_tracker.hostname), parsed_tracker.port)

    #Get connection ID
    req, transaction_id = udp_create_connection_request()
    sock.sendto(req, conn);
    buf = sock.recvfrom(2048)[0]
    connection_id = udp_parse_connection_response(buf, transaction_id)

    #Scrape away
    req, transaction_id = udp_create_scrape_request(connection_id, hashes)
    sock.sendto(req, conn)
    buf = sock.recvfrom(2048)[0]
    return udp_parse_scrape_response(buf, transaction_id, hashes)

def scrape_http(parsed_tracker, hashes):
    print "Scraping HTTP: %s for %s hashes" % (parsed_tracker.geturl(), len(hashes))
    qs = []
    for hash in hashes:
        url_param = binascii.a2b_hex(hash)
        qs.append(("info_hash", url_param))
    qs = urllib.urlencode(qs)
    pt = parsed_tracker	
    url = urlunsplit((pt.scheme, pt.netloc, pt.path, qs, pt.fragment))
    handle = urllib.urlopen(url);
    if handle.getcode() is not 200:
        raise RuntimeError("%s status code returned" % handle.getcode())
    decoded = bdecode(handle.read())
    ret = {}
    for hash, stats in decoded['files'].iteritems():
        nice_hash = binascii.b2a_hex(hash)
        s = stats["complete"]
        p = stats["incomplete"]
        c = stats["downloaded"]
        ret[nice_hash] = { "seeds" : s, "peers" : p, "complete" : c}
    return ret

def udp_create_connection_request():
    connection_id = 0x41727101980 #default connection id
    action = 0x0 #action (0 = give me a new connection id)
    transaction_id = udp_get_transaction_id()
    buf = struct.pack("!q", connection_id) #first 8 bytes is connection id
    buf += struct.pack("!i", action) #next 4 bytes is action
    buf += struct.pack("!i", transaction_id) #next 4 bytes is transaction id
    return (buf, transaction_id)

def udp_parse_connection_response(buf, sent_transaction_id):
    if len(buf) < 16:
        raise RuntimeError("Wrong response length getting connection id: %s" % len(buf))
    action = struct.unpack_from("!i", buf)[0] #first 4 bytes is action

    res_transaction_id = struct.unpack_from("!i", buf, 4)[0] #next 4 bytes is transaction id
    if res_transaction_id != sent_transaction_id:
        raise RuntimeError("Transaction ID doesnt match in connection response! Expected %s, got %s"
            % (sent_transaction_id, res_transaction_id))

    if action == 0x0:
        connection_id = struct.unpack_from("!q", buf, 8)[0] #unpack 8 bytes from byte 8, should be the connection_id
        return connection_id
    elif action == 0x3:		
        error = struct.unpack_from("!s", buf, 8)
        raise RuntimeError("Error while trying to get a connection response: %s" % error)
    pass

def udp_create_scrape_request(connection_id, hashes):
    action = 0x2 #action (2 = scrape)
    transaction_id = udp_get_transaction_id()
    buf = struct.pack("!q", connection_id) #first 8 bytes is connection id
    buf += struct.pack("!i", action) #next 4 bytes is action 
    buf += struct.pack("!i", transaction_id) #followed by 4 byte transaction id
    #from here on, there is a list of info_hashes. They are packed as char[]
    for hash in hashes:		
        hex_repr = binascii.a2b_hex(hash)
        buf += struct.pack("!20s", hex_repr)
    return (buf, transaction_id)

def udp_parse_scrape_response(buf, sent_transaction_id, hashes):
    if len(buf) < 16:
        raise RuntimeError("Wrong response length while scraping: %s" % len(buf))
    action = struct.unpack_from("!i", buf)[0] #first 4 bytes is action
    res_transaction_id = struct.unpack_from("!i", buf, 4)[0] #next 4 bytes is transaction id
    if res_transaction_id != sent_transaction_id:
        raise RuntimeError("Transaction ID doesnt match in scrape response! Expected %s, got %s"
            % (sent_transaction_id, res_transaction_id))
    if action == 0x2:
        ret = {}
        offset = 8; #next 4 bytes after action is transaction_id, so data doesnt start till byte 8
        for hash in hashes:
            seeds = struct.unpack_from("!i", buf, offset)[0]
            offset += 4
            complete = struct.unpack_from("!i", buf, offset)[0]
            offset += 4
            leeches = struct.unpack_from("!i", buf, offset)[0]
            offset += 4
            ret[hash] = { "seeds" : seeds, "peers" : leeches, "complete" : complete }
        return ret
    elif action == 0x3:
        #an error occured, try and extract the error string
        error = struct.unpack_from("!s", buf, 8)
        raise RuntimeError("Error while scraping: %s" % error)

def udp_get_transaction_id():
    return int(random.randrange(0, 255))


In [4]:
scrape("udp://tracker.leechers-paradise.org:6969",["49f3dfa4956012bf5714c1ccb13e097fd0ae484e"])

Scraping UDP: udp://tracker.leechers-paradise.org:6969 for 1 hashes


{'49f3dfa4956012bf5714c1ccb13e097fd0ae484e': {'complete': 47,
  'peers': 8,
  'seeds': 30}}

In [2]:
import binascii, urllib, socket, random, struct
from bcode import bdecode
from urlparse import urlparse, urlunsplit

import urllib2

def announce(tracker, hashes):
    """
    Returns the list of seeds, peers and downloads a torrent info_hash has, according to the specified tracker
    Args:
        tracker (str): The announce url for a tracker, usually taken directly from the torrent metadata
        hashes (list): A list of torrent info_hash's to query the tracker for
    Returns:
        A dict of dicts. The key is the torrent info_hash's from the 'hashes' parameter,
        and the value is a dict containing "seeds", "peers" and "complete".
        Eg:
        {
            "2d88e693eda7edf3c1fd0c48e8b99b8fd5a820b2" : { "seeds" : "34", "peers" : "189", "complete" : "10" },
            "8929b29b83736ae650ee8152789559355275bd5c" : { "seeds" : "12", "peers" : "0", "complete" : "290" }
        }
    """
    tracker = tracker.lower()
    parsed = urlparse(tracker)	
    if parsed.scheme == "udp":
        return announce_udp(parsed, hashes)

    if parsed.scheme in ["http", "https"]:
        if "announce" not in tracker:
            raise RuntimeError("%s doesnt support scrape" % tracker)
        parsed = urlparse(tracker.replace("announce", "scrape"))		 
        return scrape_http(parsed, hashes)

    raise RuntimeError("Unknown tracker scheme: %s" % parsed.scheme)	

def announce_udp(parsed_tracker, hashes):
    print "Scraping UDP: %s for %s hashes" % (parsed_tracker.geturl(), len(hashes))
    if len(hashes) > 74:
        raise RuntimeError("Only 74 hashes can be scraped on a UDP tracker due to UDP limitations")
    transaction_id = "\x00\x00\x04\x12\x27\x10\x19\x70";
    connection_id = "\x00\x00\x04\x17\x27\x10\x19\x80";
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    sock.settimeout(8)
    conn = (socket.gethostbyname(parsed_tracker.hostname), parsed_tracker.port)

    #Get connection ID
    req, transaction_id = udp_create_connection_request()
    sock.sendto(req, conn);
    buf = sock.recvfrom(2048)[0]
    connection_id = udp_parse_connection_response(buf, transaction_id)

    #Scrape away
    req, transaction_id = udp_create_announcer_request(connection_id, hashes)
    sock.sendto(req, conn)
    buf = sock.recvfrom(2048)[0]
    return udp_parse_announce_response(buf, transaction_id, hashes)

# def scrape_http(parsed_tracker, hashes):
#     print "Scraping HTTP: %s for %s hashes" % (parsed_tracker.geturl(), len(hashes))
#     qs = []
#     for hash in hashes:
#         url_param = binascii.a2b_hex(hash)
#         qs.append(("info_hash", url_param))
#     qs = urllib.urlencode(qs)
#     pt = parsed_tracker	
#     url = urlunsplit((pt.scheme, pt.netloc, pt.path, qs, pt.fragment))
#     handle = urllib.urlopen(url);
#     if handle.getcode() is not 200:
#         raise RuntimeError("%s status code returned" % handle.getcode())
#     decoded = bdecode(handle.read())
#     ret = {}
#     for hash, stats in decoded['files'].iteritems():
#         nice_hash = binascii.b2a_hex(hash)
#         s = stats["complete"]
#         p = stats["incomplete"]
#         c = stats["downloaded"]
#         ret[nice_hash] = { "seeds" : s, "peers" : p, "complete" : c}
#     return ret

def udp_create_connection_request():
    connection_id = 0x41727101980 #default connection id
    action = 0x0 #action (0 = give me a new connection id)
    transaction_id = udp_get_transaction_id()
    buf = struct.pack("!q", connection_id) #first 8 bytes is connection id
    buf += struct.pack("!i", action) #next 4 bytes is action
    buf += struct.pack("!i", transaction_id) #next 4 bytes is transaction id
    return (buf, transaction_id)

def udp_parse_connection_response(buf, sent_transaction_id):
    if len(buf) < 16:
        raise RuntimeError("Wrong response length getting connection id: %s" % len(buf))
    action = struct.unpack_from("!i", buf)[0] #first 4 bytes is action

    res_transaction_id = struct.unpack_from("!i", buf, 4)[0] #next 4 bytes is transaction id
    if res_transaction_id != sent_transaction_id:
        raise RuntimeError("Transaction ID doesnt match in connection response! Expected %s, got %s"
            % (sent_transaction_id, res_transaction_id))

    if action == 0x0:
        connection_id = struct.unpack_from("!q", buf, 8)[0] #unpack 8 bytes from byte 8, should be the connection_id
        return connection_id
    elif action == 0x3:	
        error = struct.unpack_from("!s", buf, 8)
        raise RuntimeError("Error while trying to get a connection response: %s" % error)
    pass

def udp_create_announcer_request(connection_id, hashes):
    action = 0x1 #action (1 = announce)
    transaction_id = udp_get_transaction_id()
    # print "2.Transaction ID :", transaction_id
    buf = struct.pack("!q", connection_id)                                  #first 8 bytes is connection id
    buf += struct.pack("!i", action)                                        #next 4 bytes is action 
    buf += struct.pack("!i", transaction_id)                                #followed by 4 byte transaction id
    for hash in hashes:
        hex_repr = binascii.a2b_hex(hash)
        buf += struct.pack("!20s", hex_repr)
    buf += struct.pack("!20s", 'ABCDEFGHIJKLMNOPQRST')          #the peer_id we announce
    buf += struct.pack("!q", 0)    #number of bytes downloaded
    buf += struct.pack("!q", 0)          #number of bytes left
    buf += struct.pack("!q", 0)      #number of bytes uploaded
    buf += struct.pack("!i", 0x2)                                           #event 2 denotes start of downloading
    buf += struct.pack("!i", 0x0)                                           #IP address set to 0. Response received to the sender of this packet
    key = udp_get_transaction_id()                                          #Unique key randomized by client
    buf += struct.pack("!i", key)
    buf += struct.pack("!i", -1)                                            #Number of peers required. Set to -1 for default
    buf += struct.pack("!i", 6881)                                        #port on which response will be sent
    return (buf, transaction_id)

def udp_parse_announce_response(buf, sent_transaction_id, hashes):
    if len(buf) < 20:
        raise RuntimeError("Wrong response length while announcing: %s" % len(buf)) 
    action = struct.unpack_from("!i", buf)[0] #first 4 bytes is action
    res_transaction_id = struct.unpack_from("!i", buf, 4)[0] #next 4 bytes is transaction id    
    if res_transaction_id != sent_transaction_id:
        raise RuntimeError("Transaction ID doesnt match in announce response! Expected %s, got %s"
            % (sent_transaction_id, res_transaction_id))
    print "Reading Response"
    if action == 0x1:
        print "Action is 3"
        ret = dict()
        offset = 8; #next 4 bytes after action is transaction_id, so data doesnt start till byte 8      
        for hash in hashes:
            interval = struct.unpack_from("!i", buf, offset)[0]
            print "Interval:"+str(interval)
            offset += 4
            leeches = struct.unpack_from("!i", buf, offset)[0]
            print "Leeches:"+str(leeches)
            offset += 4
            seeds = struct.unpack_from("!i", buf, offset)[0]
            print "Seeds:"+str(seeds)
            offset += 4
            ret[hash] = { "seeds" : seeds, "peers" : leeches, "interval" : interval }
            peers = list()
            x = 0
            while offset != len(buf):
                peers.append(dict())
                peers[x]['IP'] = struct.unpack_from("!i",buf,offset)[0]
                ip = socket.inet_ntoa(struct.pack("!i",peers[x]['IP']))
                print "IP: "+ ip
                
                #Converting IP Address to City:
                
                #response = urllib.urlopen('http://freegeoip.net/json/'+ 
                #                          str(socket.inet_ntoa(struct.pack("!i",peers[x]['IP'])))).read()
                #print response

                
                offset += 4
                if offset >= len(buf):
                    raise RuntimeError("Error while reading peer port")
                peers[x]['port'] = struct.unpack_from("!H",buf,offset)[0]
                port = str(peers[x]['port'])
                print "Port: "+port
                offset += 2
                x += 1
            return ret,peers
    else:
        #an error occured, try and extract the error string
        error = struct.unpack_from("!s", buf, 8)
        print "Action="+str(action)
        raise RuntimeError("Error while annoucing: %s" % error)
    
    

def udp_get_transaction_id():
    return int(random.randrange(0, 255))

In [3]:
announce("udp://tracker.leechers-paradise.org:6969",["89ebccd43fa8f75f8439896ea5211ea60e041a4f"])

Scraping UDP: udp://tracker.leechers-paradise.org:6969 for 1 hashes
Reading Response
Action is 3
Interval:1742
Leeches:346
Seeds:2262
IP: 200.57.102.162
Port: 53327
IP: 91.231.186.250
Port: 28675
IP: 160.152.22.100
Port: 42497
IP: 177.1.182.8
Port: 42133
IP: 195.138.86.3
Port: 57284
IP: 78.111.186.210
Port: 40930
IP: 188.255.253.185
Port: 26657
IP: 201.48.149.197
Port: 14160
IP: 46.197.133.33
Port: 34030
IP: 172.56.17.135
Port: 16591
IP: 179.127.111.98
Port: 24664
IP: 54.67.8.56
Port: 8101
IP: 144.48.110.60
Port: 12801
IP: 41.222.180.191
Port: 45682
IP: 179.212.194.106
Port: 46741
IP: 2.234.230.224
Port: 20924
IP: 5.155.154.50
Port: 41443
IP: 186.212.140.198
Port: 39685
IP: 88.145.151.222
Port: 46829
IP: 121.54.47.59
Port: 52491
IP: 187.54.211.216
Port: 31817
IP: 201.86.170.87
Port: 64470
IP: 200.103.69.227
Port: 37416
IP: 197.32.235.108
Port: 35722
IP: 189.68.23.41
Port: 48993
IP: 179.210.57.98
Port: 18362
IP: 178.157.250.238
Port: 19566
IP: 177.84.22.41
Port: 64498
IP: 156.67.121.50


({'89ebccd43fa8f75f8439896ea5211ea60e041a4f': {'interval': 1742,
   'peers': 346,
   'seeds': 2262}},
 [{'IP': -935762270, 'port': 53327},
  {'IP': 1541913338, 'port': 28675},
  {'IP': -1600645532, 'port': 42497},
  {'IP': -1325287928, 'port': 42133},
  {'IP': -1014344189, 'port': 57284},
  {'IP': 1315945170, 'port': 40930},
  {'IP': -1124074055, 'port': 26657},
  {'IP': -919562811, 'port': 14160},
  {'IP': 784696609, 'port': 34030},
  {'IP': -1405611641, 'port': 16591},
  {'IP': -1283494046, 'port': 24664},
  {'IP': 910362680, 'port': 8101},
  {'IP': -1875874244, 'port': 12801},
  {'IP': 702461119, 'port': 45682},
  {'IP': -1277902230, 'port': 46741},
  {'IP': 48948960, 'port': 20924},
  {'IP': 94083634, 'port': 41443},
  {'IP': -1160475450, 'port': 39685},
  {'IP': 1485936606, 'port': 46829},
  {'IP': 2033594171, 'port': 52491},
  {'IP': -1154034728, 'port': 31817},
  {'IP': -917067177, 'port': 64470},
  {'IP': -932755997, 'port': 37416},
  {'IP': -987698324, 'port': 35722},
  {'IP':